In [10]:
def evaluate_testset():
    # 在整个测试集上评估，返回分类评估指标日志
    
    # 交叉熵损失函数
    criterion = nn.CrossEntropyLoss() 
    loss_list = []
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_loader: # 生成一个 batch 的数据和标注
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) # 输入模型，执行前向预测
            # outputs = outputs.logits
            # 获取整个测试集的标签类别和预测类别
            _, preds = torch.max(outputs, 1) # 获得当前 batch 所有图像的预测类别
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels) # 由 logit，计算当前 batch 中，每个样本的平均交叉熵损失函数值
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)
        
    log_test = {}
    
    # 计算分类评估指标
    log_test['test_loss'] = np.mean(loss)
    log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
    log_test['test_precision'] = precision_score(labels_list, preds_list, average='macro')
    log_test['test_recall'] = recall_score(labels_list, preds_list, average='macro')
    log_test['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')
    
    return log_test

In [16]:
print(accuracy_save)
print(loss_save)
print(precision_save)
print(recall_save)
print(f1_score_save)

[1.0, 1.0, 1.0, 0.9983333333333333, 0.9976666666666667, 0.9903333333333333, 0.9743333333333334, 0.9853333333333333, 0.9653333333333334, 0.9626666666666667, 0.9136666666666666, 0.9273333333333333, 0.8803333333333333, 0.8196666666666667, 0.532, 0.3333333333333333]
[7.5420765e-05, 6.981732e-05, 7.1048555e-05, 5.9685808e-05, 0.00017323195, 0.00021742497, 0.0031757485, 0.006021244, 0.0030844714, 0.0036849945, 0.0019149194, 0.022595167, 0.05196352, 0.32560533, 0.8809403, 1.2620946]
[1.0, 1.0, 1.0, 0.9983416252072969, 0.9976828864614365, 0.9906057661159702, 0.9761683689260291, 0.9859514687100894, 0.9685990338164251, 0.9664268585131893, 0.9314270585120465, 0.9393590682428427, 0.903869815806759, 0.8320145942734257, 0.6926977309538548, 0.1111111111111111]
[1.0, 1.0, 1.0, 0.9983333333333334, 0.9976666666666666, 0.9903333333333334, 0.9743333333333334, 0.9853333333333333, 0.9653333333333333, 0.9626666666666667, 0.9136666666666667, 0.9273333333333333, 0.8803333333333333, 0.8196666666666667, 0.531999

In [28]:
from torch.optim import lr_scheduler
# 交叉熵损失函数
criterion = nn.CrossEntropyLoss()

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 将学习率设置为原来的 1/10

# 训练轮次 Epoch
EPOCHS = 5
batch_idx = 0
# 学习率降低策略
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [30]:
def train_one_batch(images, labels):
    # 运行一个 batch 的训练，返回当前 batch 的训练日志
    
    # 获得一个 batch 的数据和标注
    images = images.to(device)
    labels = labels.to(device)
    
    outputs = model(images) # 输入模型，执行前向预测
    loss = criterion(outputs, labels) # 计算当前 batch 中，每个样本的平均交叉熵损失函数值
    
    # 优化更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 获取当前 batch 的标签类别和预测类别
    _, preds = torch.max(outputs, 1) # 获得当前 batch 所有图像的预测类别
    preds = preds.cpu().numpy()
    loss = loss.detach().cpu().numpy()
    outputs = outputs.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    
    log_train = {}
    log_train['epoch'] = epoch
    log_train['batch'] = batch_idx
    # 计算分类评估指标 用于动态监测loss与accuracy
    log_train['train_loss'] = loss
    log_train['train_accuracy'] = accuracy_score(labels, preds)
    # log_train['train_precision'] = precision_score(labels, preds, average='macro')
    # log_train['train_recall'] = recall_score(labels, preds, average='macro')
    # log_train['train_f1-score'] = f1_score(labels, preds, average='macro')
    
    return log_train

In [33]:
# 载入剪枝后的模型作为当前模型
model = torch.load('pruned_resnet50.pth')
model = model.to(device)

# 微调模型
for epoch in range(1, EPOCHS+1):
    
    print(f'Epoch {epoch}/{EPOCHS}')  # 打印当前训练轮数
    
    ## 训练阶段
    model.train()
    for images, labels in tqdm(train_loader):  # 获得一个 batch 的数据和标注
        batch_idx += 1
        log_train = train_one_batch(images, labels)  # 一次训练一个batch
        df_train_log = df_train_log.append(log_train, ignore_index=True)
        
    lr_scheduler.step()

    ## 测试阶段
    model.eval()
    log_test = evaluate_testset()
    print(log_test)
    df_test_log = df_test_log.append(log_test, ignore_index=True)
    
    if log_test['test_accuracy'] >= best_test_accuracy: 
        if log_test['test_loss'] < best_test_loss: 
            # 删除旧的最佳模型文件(如有)
            old_best_checkpoint_path = './fine_tuned_pruned_resnet50-{:.3f}.pth'.format(best_test_accuracy)
            if os.path.exists(old_best_checkpoint_path):
                os.remove(old_best_checkpoint_path)
            # 保存新的最佳模型文件
            new_best_checkpoint_path = './fine_tuned_pruned_resnet50-{:.3f}.pth'.format(log_test['test_accuracy'])
            torch.save(model, new_best_checkpoint_path)
            print('保存新的最佳模型', './fine_tuned_pruned_resnet50-{:.3f}.pth'.format(best_test_accuracy))
            best_test_accuracy = log_test['test_accuracy']
            best_test_loss = log_test['test_loss']

# # 保存微调后的模型
# torch.save(model, 'fine_tuned_pruned_resnet50-{:.3f}.pth')

# # 评估微调后的模型
# model.eval()
# print(evaluate_testset())

Epoch 1/5


100%|██████████| 188/188 [00:38<00:00,  4.92it/s]


{'test_loss': 0.0012905959, 'test_accuracy': 0.9993333333333333, 'test_precision': 0.9993346640053226, 'test_recall': 0.9993333333333334, 'test_f1-score': 0.9993333326666659}
保存新的最佳模型 ./fine_tuned_pruned_resnet50-0.000.pth
Epoch 2/5


100%|██████████| 188/188 [00:38<00:00,  4.93it/s]


{'test_loss': 0.0012217328, 'test_accuracy': 0.9993333333333333, 'test_precision': 0.9993346640053226, 'test_recall': 0.9993333333333334, 'test_f1-score': 0.9993333326666659}
保存新的最佳模型 ./fine_tuned_pruned_resnet50-0.999.pth
Epoch 3/5


...

{'test_loss': 0.0009464224, 'test_accuracy': 0.9993333333333333, 'test_precision': 0.9993346640053226, 'test_recall': 0.9993333333333334, 'test_f1-score': 0.9993333326666659}
保存新的最佳模型 ./fine_tuned_pruned_resnet50-0.999.pth
Epoch 4/5


100%|██████████| 188/188 [00:38<00:00,  4.94it/s]


{'test_loss': 0.0012235055, 'test_accuracy': 0.9993333333333333, 'test_precision': 0.9993346640053226, 'test_recall': 0.9993333333333334, 'test_f1-score': 0.9993333326666659}
Epoch 5/5


100%|██████████| 188/188 [00:37<00:00,  4.97it/s]


{'test_loss': 0.0014658093, 'test_accuracy': 0.9993333333333333, 'test_precision': 0.9993346640053226, 'test_recall': 0.9993333333333334, 'test_f1-score': 0.9993333326666659}


In [35]:
# 评估微调后的模型
model = torch.load('fine_tuned_pruned_resnet50-0.999.pth')
model.eval()
print(evaluate_testset())

{'test_loss': 0.0009464224, 'test_accuracy': 0.9993333333333333, 'test_precision': 0.9993346640053226, 'test_recall': 0.9993333333333334, 'test_f1-score': 0.9993333326666659}
